In [1]:
from scipy.stats import ttest_ind
from sklearn.metrics import roc_auc_score

In [2]:
import pandas as pd

In [3]:
df_5e6_fram = pd.read_csv('/legodata/zhaoj/cvd_risk_time2/src/dnn/results/exp1/5e6_fram_pred.csv')
df_5e7_fram = pd.read_csv('/legodata/zhaoj/cvd_risk_time2/src/dnn/results/exp1/5e7_fram_pred.csv')
df_5e8_fram = pd.read_csv('/legodata/zhaoj/cvd_risk_time2/src/dnn/results/exp1/5e8_fram_pred.csv')
df_5e10_fram = pd.read_csv('/legodata/zhaoj/cvd_risk_time2/src/dnn/results/exp1/5e10_fram_pred.csv')

df_5e6_full = pd.read_csv('/legodata/zhaoj/cvd_risk_time2/src/dnn/results/exp1/5e6_full_pred.csv')
df_5e7_full = pd.read_csv('/legodata/zhaoj/cvd_risk_time2/src/dnn/results/exp1/5e7_full_pred.csv')
df_5e8_full = pd.read_csv('/legodata/zhaoj/cvd_risk_time2/src/dnn/results/exp1/5e8_full_pred.csv')
df_5e10_full = pd.read_csv('/legodata/zhaoj/cvd_risk_time2/src/dnn/results/exp1/5e10_full_pred.csv')

full_dict = {'5e6': df_5e6_full, '5e7': df_5e7_full, '5e8': df_5e8_full, '5e10': df_5e10_full}
fram_dict = {'5e6': df_5e6_fram, '5e7': df_5e7_fram, '5e8': df_5e8_fram, '5e10': df_5e10_fram}


In [4]:
for df_pred in [df_5e6_fram, df_5e7_fram, df_5e8_fram, df_5e10_fram]:
    print(roc_auc_score(df_pred['CLASS'], df_pred['SCORE']))

0.557379796415
0.561286826806
0.557841448459
0.558858852001


In [5]:
dict_summary = dict() 

In [6]:
for thresh in ['5e6', '5e7', '5e8', '5e10']:
    avg_auc = 0 
    df_pred = fram_dict[thresh]
    
    dict_summary['{}_fram'.format(thresh)] = list()
    
    for i in range(10): 
        df_pred_fold = df_pred.loc[df_pred['FOLD'] == i]
        roc_auc = roc_auc_score(df_pred_fold['CLASS'], df_pred_fold['SCORE'])
        
        dict_summary['{}_fram'.format(thresh)].append(roc_auc)
        avg_auc += roc_auc 
    
    print(avg_auc / 10)

0.560819990626
0.5628638653
0.558959294848
0.559416942359


In [7]:
for thresh in ['5e6', '5e7', '5e8', '5e10']:
    avg_auc = 0 
    df_pred = full_dict[thresh]
    
    dict_summary['{}_full'.format(thresh)] = list()
    
    for i in range(10): 
        df_pred_fold = df_pred.loc[df_pred['FOLD'] == i]
        roc_auc = roc_auc_score(df_pred_fold['CLASS'], df_pred_fold['SCORE'])
        
        dict_summary['{}_full'.format(thresh)].append(roc_auc)
        avg_auc += roc_auc 
    
    print(avg_auc / 10)

0.541968895882
0.541173377802
0.546572086245
0.543700489788


In [11]:
df_summary = pd.DataFrame(dict_summary)
df_summary.head(10)

import statistics 

for col in list(df_summary.columns): 
    auc_list = df_summary[col]
    print(col)
    print(statistics.mean(auc_list))
    print(statistics.stdev(auc_list))
    

5e10_fram
0.559416942359
0.012001811252349975
5e10_full
0.543700489788
0.0046131792041774625
5e6_fram
0.560819990626
0.014170922999063057
5e6_full
0.541968895882
0.011213481757557523
5e7_fram
0.5628638653
0.012907412502617859
5e7_full
0.541173377802
0.009243803694011685
5e8_fram
0.558959294848
0.01043929919755111
5e8_full
0.546572086245
0.01247789711905685


In [ ]:
import os 

df_summary.to_csv(os.path.join('/legodata/zhaoj/cvd_risk_time2/src/dnn/summary.csv'))

In [15]:
for df_1, df_2 in [(df_5e6_full, df_5e6_fram)]: 
    auc_dict = {'df_1': [], 'df_2': []} 
    
    df_2_avg = 0 
    df_1_avg = 0 
    
    for i in range(10): 
        
        df_2_fold = df_1.loc[df_1['FOLD'] == i]
        df_1_fold = df_2.loc[df_2['FOLD'] == i] 
        df_1_auc = roc_auc_score(df_1_fold['CLASS'], df_1_fold['SCORE'])
        df_2_auc = roc_auc_score(df_2_fold['CLASS'], df_2_fold['SCORE']) 
        
        df_1_avg += df_1_auc 
        df_2_avg += df_2_auc 
        
        auc_dict['df_1'].append(df_1_auc)
        auc_dict['df_2'].append(df_2_auc) 
        
    df_1_avg /= 10 
    df_2_avg /= 10 
    
    print(df_1_avg)
    print(df_2_avg)
    
    df_auc = pd.DataFrame(auc_dict)
    
    print(ttest_ind(auc_dict['df_1'], auc_dict['df_2']))

        

0.560819990626
0.541968895882
Ttest_indResult(statistic=3.2988071197073334, pvalue=0.0039918037930929883)


In [8]:
for full, fram in [(gbt_full, gbt_true_fram), (lr_full, lr_true_fram), (rf_full, rf_true_fram)]: 
    auc_dict = {'full': [], 'fram': []} 
    
    fram_avg = 0 
    full_avg = 0 
    
    for i in range(10): 
        fram_fold = full.loc[full['FOLD'] == i]
        full_fold = fram.loc[fram['FOLD'] == i] 
        full_auc = roc_auc_score(full_fold['CLASS'], full_fold['SCORE'])
        fram_auc = roc_auc_score(fram_fold['CLASS'], fram_fold['SCORE']) 
        
        full_avg += full_auc 
        fram_avg += fram_auc 
        
        auc_dict['full'].append(full_auc)
        auc_dict['fram'].append(fram_auc) 
        
    full_avg /= 10 
    fram_avg /= 10 
    
    print(full_avg)
    print(fram_avg)
    
    df_auc = pd.DataFrame(auc_dict)
    
    print(ttest_ind(auc_dict['full'], auc_dict['fram']))

        

NameError: name 'gbt_full' is not defined